In [1]:
# Import necessary modules

import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from pathlib import Path
import xarray as xr
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from datetime import datetime
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
from cartopy.util import add_cyclic_point
import matplotlib.colors as colors
import geopandas as gpd
from urllib.request import urlretrieve
import requests
from glob import glob
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from pathlib import Path
from datetime import datetime, timedelta
import json
import sys
import math
from geopy.distance import distance
from matplotlib.colors import to_rgba
from collections import defaultdict

In [2]:

parent_dir    = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_dir      = os.path.join(parent_dir, 'data')
satellite_dir = os.path.join(data_dir, 'satellite')
output_dir    = os.path.join(parent_dir, 'Output/sat_plot')
NEODASS_dir   = os.path.join(satellite_dir, 'NEODASS')
floats_dir    = os.path.join(parent_dir, 'Data/Floats')

In [17]:
# Load the netCDF file
file      = 'CMEMS_cmems_mod_glo_phy-cur_anfc_0.083deg_P1D-m_April-May2024.nc'
filepath  = os.path.join(satellite_dir, file)

# Load the netCDF file
ds = xr.open_dataset(filepath)

# Extract the relevant data
depth = ds['depth']
uo = ds['uo']
vo = ds['vo']

# Ensure depth values are unique
depth_unique, index = np.unique(depth, return_index=True)
uo_unique = uo.isel(depth=index)
vo_unique = vo.isel(depth=index)
depth_diff = np.diff(depth_unique, append=depth_unique[-1])

# Expand depth_diff to match the dimensions of uo and vo
depth_diff_da = xr.DataArray(depth_diff, coords={'depth': depth_unique}, dims='depth')
depth_diff_expanded = depth_diff_da.broadcast_like(uo_unique)

# Compute depth-weighted averages
def weighted_avg(da, weights):
    return (da * weights).sum(dim='depth') / weights.sum(dim='depth')

# Apply the weighted average function
weighted_uo = weighted_avg(uo_unique, depth_diff_expanded)
weighted_vo = weighted_avg(vo_unique, depth_diff_expanded)

# Create a new dataset with the averaged data
averaged_ds = xr.Dataset({
    'uo': weighted_uo,
    'vo': weighted_vo
})

# Add attributes
averaged_ds['uo'].attrs = uo.attrs
averaged_ds['vo'].attrs = vo.attrs
averaged_ds.attrs = ds.attrs

# Save the averaged data to a new netCDF file
output_file      = 'CMEMS_cmems_mod_glo_phy-cur_nfc_0.083deg_P1D-m_April-May2024_averaged.nc'
output_file_path = os.path.join(satellite_dir, output_file)
averaged_ds.to_netcdf(output_file_path)

In [22]:
### Plotting most recent data from CMEMS
## List most recent files

# List to store matching file names
matching_files = []

# Loop through the files in the directory
for filename in os.listdir(satellite_dir):
    if filename.endswith('.nc'):
        matching_files.append(filename)

print(matching_files)

## List variables from these files
# Dictionary to store files by their base names (excluding '_a_' or '_b_') and key variables
file_dict = {}

for file in matching_files:
    # Identify the base name by removing '_a' or '_b' if present
    base_name = file
    if base_name not in file_dict:
        file_dict[base_name] = {}
    filepath = os.path.join(satellite_dir, file)
    ds = xr.open_dataset(filepath)
    variables = list(ds.data_vars.keys())

    print(file)
    print(variables)

    key_variables = [var for var in variables if 'longitude' not in var.lower() and 'latitude' not in var.lower()]

    if key_variables:
        for key_variable in key_variables:
            if file not in file_dict[base_name]:
                file_dict[base_name][file] = {}
            file_dict[base_name][file][key_variable] = {
                'variable': key_variable,
            }
    else:
        print(f"No suitable key variable found in file: {file}")

    ds.close()

# Print the results
for base_name, files in file_dict.items():
    for file, key_variable in files.items():
        print(f'File: {file}')
        print(f'Base Name: {base_name}')
        print(f'Key Variable: {key_variable}')

    filepath  = os.path.join(satellite_dir, file)
    dataset = xr.open_dataset(filepath)

    # Print global attributes
    print("Global attributes:")
    for attr_name, attr_value in dataset.attrs.items():
        print(f"  {attr_name}: {attr_value}")

    # Print variable attributes
    print("\nVariable attributes:")
    for var_name, var in dataset.variables.items():
        print(f"Variable: {var_name}")
        for attr_name, attr_value in var.attrs.items():
            print(f"  {attr_name}: {attr_value}")

# Initialize var_dict with categories as keys and empty dictionaries as values
var_dict = {'chl': {}, 'temp': {}, 'sla': {}, 'adt': {}, 'bbp': {}, 'dissic': {}, 'ugos':{}, 'uo':{}} 

for base_name, files in file_dict.items():
    for file, variables in files.items():
        contains_adt = False
        contains_ugos = False
        contains_uo = False
        
        for key_variable, details in variables.items():
            data_var = details['variable']
            var_name_lower = data_var.lower()
            
            if 'chl' in var_name_lower and var_name_lower == 'chl':
                var_dict['chl'][file] = data_var
            elif 'temp' in var_name_lower:
                var_dict['temp'][file] = data_var
            elif 'sla' in var_name_lower:
                var_dict['sla'][file] = data_var
            elif 'adt' in var_name_lower:
                var_dict['adt'][file] = data_var
                contains_adt = True
            elif 'bbp' in var_name_lower and var_name_lower == 'bbp':
                var_dict['bbp'][file] = data_var
            elif 'dissic' in var_name_lower and var_name_lower == 'dissic':
                var_dict['dissic'][file] = data_var
            elif 'ugos' in var_name_lower and var_name_lower == 'ugos':
                contains_ugos = True
            elif 'uo' in var_name_lower and var_name_lower == 'uo' and 'averaged' in base_name:
                contains_uo = True
                # Delay adding 'ugos' to var_dict until after the adt check

        # After checking all variables in the file
        if contains_ugos:
            if contains_adt:
                # Add to 'adt' category if the file contains both 'adt' and 'ugos'
                var_dict['adt'][file] = data_var
            else:
                # Add to 'ugos' category if the file contains only 'ugos'
                var_dict['ugos'][file] = data_var
        if contains_uo:
            var_dict['uo'][file] = data_var


['CMEMS_cmems_mod_glo_bgc-car_anfc_0.25deg_P1D-m_April-May2024.nc', 'CMEMS_cmems_mod_glo_phy-cur_anfc_0.083deg_P1D-m_April-May2024.nc', 'CMEMS_cmems_mod_glo_phy-cur_nfc_0.083deg_P1D-m_April-May2024_averaged.nc', 'CMEMS_cmems_obs-oc_atl_bgc-optics_nrt_l3-multi-1km_P1D_April-May2024.nc', 'CMEMS_cmems_obs-oc_atl_bgc-plankton_nrt_l3-multi-1km_P1D_April-May2024.nc', 'CMEMS_cmems_obs-sl_eur_phy-ssh_nrt_allsat-l4-duacs-0.125deg_P1D_April-May2024.nc', 'CMEMS_cmems_obs_mob_glo_phy-cur_nrt_0.25deg_P1D-m_April-May2024.nc']
CMEMS_cmems_mod_glo_bgc-car_anfc_0.25deg_P1D-m_April-May2024.nc
['dissic', 'ph', 'talk']
CMEMS_cmems_mod_glo_phy-cur_anfc_0.083deg_P1D-m_April-May2024.nc
['uo', 'vo']
CMEMS_cmems_mod_glo_phy-cur_nfc_0.083deg_P1D-m_April-May2024_averaged.nc
['uo', 'vo']
CMEMS_cmems_obs-oc_atl_bgc-optics_nrt_l3-multi-1km_P1D_April-May2024.nc
['BBP', 'BBP_uncertainty', 'CDM', 'CDM_uncertainty', 'flags']
CMEMS_cmems_obs-oc_atl_bgc-plankton_nrt_l3-multi-1km_P1D_April-May2024.nc
['CHL', 'CHL_uncertai